# Lecture 3-10: Image Measurement and Feature Extraction

## 0.- Initialize filesystem and libraries

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2

In [ ]:
!pip install pydicom
import pydicom

## 1.- Geometric measurements

In [ ]:
# Load the grayscale image
image_path = '/content/drive/MyDrive/PIM/Images/X-ray_4.png'
image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

plt.figure(figsize=(5, 5))
plt.imshow(image, cmap='gray')
plt.title('Original image')
plt.show()

In [ ]:
# Apply Otsu's thresholding
otsu_threshold, otsu_threshold_image = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
print(f"Optimal threshold (Otsu's method): {otsu_threshold}")

plt.figure(figsize=(10, 4))

plt.subplot(1, 3, 1)
plt.hist(image.flatten(), bins=256, range=[0, 256])
plt.title("Histogram of the original image\n (Otsu's threshold is marked with the red line)")
plt.xlabel('Pixel intensity')
plt.ylabel('Frequency')
plt.axvline(otsu_threshold, color='r')  # Otsu's threshold

plt.subplot(1, 3, 2)
plt.imshow(otsu_threshold_image, cmap='gray')
plt.title("Otsu's threshold")

plt.subplot(1, 3, 3)
plt.imshow(otsu_threshold_image[70:170, 80:180], cmap='gray')
plt.title("Otsu's threshold (cropped)")
plt.axis('off')

plt.tight_layout()
plt.show()

# Label connected components
num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(otsu_threshold_image, connectivity=8)
print(f"Number of labels (including background): {num_labels}")

In [ ]:
# Perform opening and closing
structuring_element = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
opened_image = cv2.morphologyEx(otsu_threshold_image, cv2.MORPH_OPEN, structuring_element)
closed_image = cv2.morphologyEx(opened_image, cv2.MORPH_CLOSE, structuring_element)

# Label connected components
num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(closed_image, connectivity=8)
print(f"Number of labels (including background): {num_labels}")

plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.imshow(opened_image, cmap='gray')
plt.title("Opened image")
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(closed_image, cmap='gray')
plt.title("Closed image")
plt.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# Label connected components
num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(closed_image, connectivity=8)

# Print the number of labels in the image
print(f"Number of labels (including background): {num_labels}")

# Print unique labels
unique_labels = np.unique(labels) # returns the sorted unique elements from the input array
print("Unique labels in the image:", unique_labels)

# Plot each labeled object separately
plt.figure(figsize=(10, 10))
for label in range(1, num_labels):  # starts from 1 to skip the background
    # Create a binary mask for the current label
    mask = np.where(labels == label, 255, 0).astype(np.uint8)
    # for each element in the array 'labels', if the condition (labels == label)
    # is True, it replaces it with 255 (white), and if False, it replaces it
    # with 0 (black) in an image). Then, it converts the resulting array to uint8 format

    # Plot each object in a separate subplot
    plt.subplot(1, num_labels-1, label)
    plt.imshow(mask, cmap='gray')
    plt.title(f'Object {label}')
    plt.axis('off')

    # Print statistics for each object:
    print(f'Object: {label}')
    print(f'  Bounding Box: x={stats[label, 0]}, y={stats[label, 1]}, width={stats[label, 2]}, height={stats[label, 3]}')
    print(f'  Area: {stats[label, 4]} pixels')
    print(f'  Centroid: (x={centroids[label, 0]:.2f}, y={centroids[label, 1]:.2f})')

plt.tight_layout()
plt.show()

In [ ]:
# Convert the original image to color image
contour_image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)

# Convert the black & white image to color image
mask_image = cv2.cvtColor(closed_image, cv2.COLOR_GRAY2RGB)

for label in range(1, num_labels):  # starts from 1 to skip the background
    # Extracts statistics for each label (x, y, width, height, area)
    x = stats[label, cv2.CC_STAT_LEFT]  # similar to stats[label, 0]
    y = stats[label, cv2.CC_STAT_TOP]  # similar to stats[label, 1]
    width = stats[label, cv2.CC_STAT_WIDTH]    # similar to stats[label, 2]
    height = stats[label, cv2.CC_STAT_HEIGHT]   # similar to stats[label, 3]
    area = stats[label, cv2.CC_STAT_AREA]    # similar to stats[label, 4]
    centroid = centroids[label]  # Centroid of each object

    # Extract the contour for the labeled region
    mask = np.uint8(np.zeros_like(image)) # create an empty mask (black image with the same dimensions as the original image)
    mask[labels == label] = 255  # create a boolean mask (in white) for each object or label
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) # find the contours
    if contours: # if a contour was extracted, then draw it
        cv2.drawContours(contour_image, contours, -1, (0, 0, 255), 2)  # Draw contours in blue

    # Extract the perimeter of the current contour
    perimeter = cv2.arcLength(contours[0], True)

    # Draw the bounding box in green (0, 255, 0) with thickness of 2 pixels
    cv2.rectangle(mask_image, (x, y), (x + width, y + height), (0, 255, 0), 2)
    # Label the object
    cv2.putText(mask_image, f"Object {label}", (int(centroids[label][0]), int(centroids[label][1])),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2) # Red color, thickness = 2

    # Calculate different shape descriptors
    compactness = (perimeter**2) / area if area != 0 else 0  # Compactness
    circularity = (4 * np.pi * area) / (perimeter**2) if perimeter != 0 else 0  # Circularity
    aspect_ratio = float(width) / height  # Aspect Ratio
    extent = float(area) / (width * height) # Extent

    print(f'Label: {label}')
    print(f'  Bounding box: x={x}, y={y}, width={width}, height={height}')
    print(f'  Centroid: (x={centroid[0]:.2f}, y={centroid[1]:.2f})')
    print(f'  Area: {area} pixels')
    print(f'  Perimeter: {perimeter:.2f} pixels')
    print(f"  Compactness: {compactness}")
    print(f"  Circularity: {circularity}")
    print(f'  Aspect ratio: {aspect_ratio:.2f}')
    print(f'  Extent: {extent:.2f}')

plt.figure(figsize=(10, 5))

# Mask image (binarized)
plt.subplot(1, 2, 1)
plt.imshow(mask_image, cmap='gray')
plt.title('Mask image (binarized)\n showing the bounding boxes')
plt.axis('off')

plt.subplot(1,2,2)
plt.imshow(contour_image)
plt.title("Object contours")
plt.axis('off')

plt.show()

In [ ]:
# Open the DICOM file
dicom_file = pydicom.dcmread('/content/drive/MyDrive/PIM/Images/IM000015.dcm')
image = dicom_file.pixel_array

# Normalize the image to the uint8 range (0,255)
image_norm = cv2.normalize(image, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)

# Binarize the image using a threshold
threshold = 20
_, binary_image = cv2.threshold(image_norm, threshold, 255, cv2.THRESH_BINARY)

# Perform closing
structuring_element = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (9, 9))
closed_image = np.uint8(cv2.morphologyEx(binary_image, cv2.MORPH_CLOSE, structuring_element))

plt.figure(figsize=(10, 5))

plt.subplot(1, 3, 1)
plt.imshow(image_norm, cmap='gray')
plt.title("DICOM image")
plt.axis('off')

plt.subplot(1, 3, 2)
plt.imshow(binary_image, cmap='gray')
plt.title("Binary image")
plt.axis('off')

plt.subplot(1, 3, 3)
plt.imshow(closed_image, cmap='gray')
plt.title("Closed image")
plt.axis('off')

plt.tight_layout()
plt.show()

# Label connected components
num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(closed_image, connectivity=8)

# Print the number of labels in the image
print(f"Number of labels (including background): {num_labels}")

# Print unique labels
unique_labels = np.unique(labels) # returns the sorted unique elements from the input array
print("Unique labels in the image:", unique_labels)

# Convert the original image to color image
contour_image = cv2.cvtColor(image_norm, cv2.COLOR_GRAY2RGB)

# Convert the black & white image to color image
mask_image = cv2.cvtColor(closed_image, cv2.COLOR_GRAY2RGB)

for label in range(1, num_labels):  # starts from 1 to skip the background
    # Extracts statistics for each label (x, y, width, height, area)
    x = stats[label, cv2.CC_STAT_LEFT]  # similar to stats[label, 0]
    y = stats[label, cv2.CC_STAT_TOP]  # similar to stats[label, 1]
    width = stats[label, cv2.CC_STAT_WIDTH]    # similar to stats[label, 2]
    height = stats[label, cv2.CC_STAT_HEIGHT]   # similar to stats[label, 3]
    area = stats[label, cv2.CC_STAT_AREA]    # similar to stats[label, 4]
    centroid = centroids[label]  # Centroid of each object

    # Extract the contour for the labeled region
    mask = np.uint8(np.zeros_like(image_norm)) # create an empty mask (black image with the same dimensions as the original image)
    mask[labels == label] = 255  # create a boolean mask (in white) for each object or label
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) # find the contours
    if contours: # if a contour was extracted, then draw it
        cv2.drawContours(contour_image, contours, -1, (0, 0, 255), 2)  # Draw contours in blue

    # Extract the perimeter of the current contour
    perimeter = cv2.arcLength(contours[0], True)

    # Draw the bounding box in green (0, 255, 0) with thickness of 2 pixels
    cv2.rectangle(mask_image, (x, y), (x + width, y + height), (0, 255, 0), 2)
    # Label the object
    cv2.putText(mask_image, f"Object {label}", (int(centroids[label][0]), int(centroids[label][1])),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2) # Red color, thickness = 2

    # Calculate different shape descriptors
    compactness = (perimeter**2) / area if area != 0 else 0  # Compactness
    circularity = (4 * np.pi * area) / (perimeter**2) if perimeter != 0 else 0  # Circularity
    aspect_ratio = float(width) / height  # Aspect Ratio
    extent = float(area) / (width * height) # Extent

    print(f'Label: {label}')
    print(f'  Bounding box: x={x}, y={y}, width={width}, height={height}')
    print(f'  Centroid: (x={centroid[0]:.2f}, y={centroid[1]:.2f})')
    print(f'  Area: {area} pixels')
    print(f'  Perimeter: {perimeter:.2f} pixels')
    print(f"  Compactness: {compactness}")
    print(f"  Circularity: {circularity}")
    print(f'  Aspect ratio: {aspect_ratio:.2f}')
    print(f'  Extent: {extent:.2f}')

plt.figure(figsize=(10, 5))

# Mask image (binarized)
plt.subplot(1, 2, 1)
plt.imshow(mask_image, cmap='gray')
plt.title('Mask image (binarized)\n showing the bounding boxes')
plt.axis('off')

plt.subplot(1,2,2)
plt.imshow(contour_image)
plt.title("Object contours")
plt.axis('off')

plt.show()


## Calculate real dimensions

# Access the Pixel Spacing attribute
pixel_spacing = dicom_file.PixelSpacing
print("Pixel spacing: ", pixel_spacing)

# Image dimensions in pixels
image_width_pixels = dicom_file.Columns
image_height_pixels = dicom_file.Rows

print("Image width (pixels): ", image_width_pixels)
print("Image height (pixels): ", image_height_pixels)

# Calculate real dimensions of the image
image_width_mm = image_width_pixels * pixel_spacing[0]
image_height_mm = image_height_pixels * pixel_spacing[1]

print("Image width (mm): ", image_width_mm)
print("Image height (mm): ", image_height_mm)

# Width and heigth of the head section in pixels
head_width_pixels = stats[label, cv2.CC_STAT_WIDTH]
head_height_pixels = stats[label, cv2.CC_STAT_HEIGHT]

print("Head width (pixels): ", head_width_pixels)
print("Head height (pixels): ", head_height_pixels)

# Calculate real dimensions of the head section (width, heigth)
head_width_mm = head_width_pixels * pixel_spacing[0]
head_height_mm = head_height_pixels * pixel_spacing[1]

print("Head width (mm): ", head_width_mm)
print("Head height (mm): ", head_height_mm)

## 2.- Corner Detection

In [ ]:
# Load the grayscale image
image_path = '/content/drive/MyDrive/PIM/Images/X-ray_4_mod.png'
image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

# Apply Harris corner detection
dst = cv2.cornerHarris(image, blockSize=2, ksize=3, k=0.04)

# Result is dilated to mark the corners
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
dst = cv2.dilate(dst, kernel, iterations=2)

# Create a copy of the image to mark the corners
image_with_corners = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)

# Threshold to mark the corners in the image using a Boolean mask
image_with_corners[dst > 0.1 * dst.max()] = [255, 0, 0]


plt.figure(figsize=(10, 5))

plt.subplot(1, 3, 1)
plt.imshow(image, cmap='gray')
plt.title('Original image')
plt.axis('off')

plt.subplot(1, 3, 2)
plt.imshow(dst, cmap='gray')
plt.title('Harris corner distance array')
plt.axis('off')

plt.subplot(1, 3, 3)
plt.imshow(image_with_corners)
plt.title('Harris corner detection')
plt.axis('off')

plt.tight_layout()
plt.show()

## 3.- Keypoints Detection

In [ ]:
# Load the grayscale image
image_path = '/content/drive/MyDrive/PIM/Images/X-ray_4.png'
image1 = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

# Function to apply rotation, scaling, and translation
def transform_image(image, angle, scale, tx, ty):
    # Get image dimensions
    rows, cols = image.shape

    # Compute the center of the image
    center = (cols // 2, rows // 2)

    # Create the transformation matrix for rotation and scaling
    M_rotation = cv2.getRotationMatrix2D(center, angle, scale)

    # Apply the rotation and scaling
    rotated_scaled_image = cv2.warpAffine(image, M_rotation, (cols, rows))

    # Create the translation matrix
    M_translation = np.float32([[1, 0, tx], [0, 1, ty]])

    # Apply the translation
    transformed_image = cv2.warpAffine(rotated_scaled_image, M_translation, (cols, rows))

    return transformed_image

# Selectable parameters for transformation
angle = 45  # Rotation angle in degrees
scale = 0.8  # Scaling factor
tx, ty = 100, 50  # Translation along the x and y axes

# Apply the transformation
image2 = transform_image(image1, angle, scale, tx, ty)


## Apply the SIFT algorithm

# 1. Initialize SIFT detector
sift = cv2.SIFT_create(nOctaveLayers=3, contrastThreshold=0.1, edgeThreshold=10)

# 2. Detect keypoints and descriptors with SIFT
# 'None' to detect keypoints in the entire image; otherwise, use a Boolean mask (255/0)
keypoints1, descriptors1 = sift.detectAndCompute(image1, None)
keypoints2, descriptors2 = sift.detectAndCompute(image2, None)

print("\nLength of keypoints in image 1: ", len(keypoints1))
print("Shape of descriptors in image 1: ", descriptors1.shape)
print("\nLength of keypoints in image 2: ", len(keypoints2))
print("Shape of descriptors in image 2: ", descriptors2.shape)

# Print the identified keypoints
#print("\nCoordinates of keypoints in image 1:")
#for kp in keypoints1:
#    print(f"({kp.pt[0]:.2f}, {kp.pt[1]:.2f})")

#print("\nCoordinates of keypoints in image 2:")
#for kp in keypoints2:
#    print(f"({kp.pt[0]:.2f}, {kp.pt[1]:.2f})")

# 3. Draw keypoints on the images
image1_keypoints = cv2.drawKeypoints(image1, keypoints1, None, color=(0, 255, 0), flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
image2_keypoints = cv2.drawKeypoints(image2, keypoints2, None, color=(0, 255, 0), flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

# Plot the images with keypoints
plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.imshow(image1_keypoints, cmap='gray')
plt.title("Keypoints in image 1")
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(image2_keypoints, cmap='gray')
plt.title("Keypoints in image 2")
plt.axis('off')

plt.show()

# 4a. Use the Brute-Force Matcher to match keypoints based on their descriptors
bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)
matches = bf.match(descriptors1, descriptors2)

# 4b. Sort the matches based on their distances (lower distance is better)
def get_distance(match): # this function receives an object, 'match', which have a property called 'distance'
    return match.distance # accesses and returns the 'distance' attribute of the 'match' object

# sorts the 'matches' list using the sorted function
# sorted() returns a new sorted list and leaves the original list unchanged
# key=get_distance indicates the sorted() function to use the get_distance function to extract
#   the value to sort by (in this case, the distance attribute)
matches = sorted(matches, key=get_distance)
print("Number of matches: ", len(matches))

# 5. Draw the matched keypoints between the two images
# Plotting of the best matches (ordered by distance)
matched_image = cv2.drawMatches(image1, keypoints1, image2, keypoints2, matches[:10], None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

# Plot the matched keypoints
plt.figure(figsize=(10, 5))
plt.imshow(matched_image)
plt.title("Matched keypoints")
plt.axis('off')
plt.show()

## 4.- Shape Detection

In [ ]:
from skimage.filters import frangi

# Load the image in grayscale
image_path = '/content/drive/MyDrive/PIM/Images/Retina_blood_vessel_9.png'
#image_path = '/content/drive/MyDrive/PIM/Images/Cerebral_Angiogram_Lateral_Wikipedia.jpg'
image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
image_float = np.float32(image)

# Apply CLAHE algorithm
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
clahe_image = clahe.apply(image)

# Convert the CLAHE image to float and normalize it to the range [0, 1]
clahe_image_float = cv2.normalize(clahe_image.astype(np.float32), None, 0.0, 1.0, cv2.NORM_MINMAX)

# Apply the Frangi filter
frangi_filtered = frangi(clahe_image_float, gamma=15, black_ridges=True)
print(f"Minimum and maximum values of the Frangi image: {np.min(frangi_filtered)}, {np.max(frangi_filtered)}")

# Scale the Frangi output back to [0, 255] and convert it to uint8
scaled_frangi = cv2.normalize(frangi_filtered, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)

# Apply histogram equalization
equalized_frangi = cv2.equalizeHist(scaled_frangi)


plt.figure(figsize=(8, 12))

# Display the original image
plt.subplot(3, 2, 1)
plt.title("Original image")
plt.imshow(image, cmap='gray', vmin=0, vmax=255)
plt.axis('off')

# Display the CLAHE-enhanced image
plt.subplot(3, 2, 2)
plt.title("CLAHE-enhanced image")
plt.imshow(clahe_image, cmap='gray', vmin=0, vmax=255)
plt.axis('off')

# Display the normalized Frangi filtered image
plt.subplot(3, 2, 3)
plt.title("Frangi filtered image (normalized)")
plt.imshow(scaled_frangi, cmap='gray', vmin=0, vmax=255)
plt.axis('off')

# Display the histogram-equalized Frangi filtered image
plt.subplot(3, 2, 4)
plt.title(f"Histogram equalized image")
plt.imshow(equalized_frangi, cmap='gray', vmin=0, vmax=255)
plt.axis('off')

# Display the histogram of the Frangi filtered image
plt.subplot(3, 2, 5)
plt.hist(scaled_frangi.flatten(), bins=256, range=[0, 256], color='black')
plt.title(f'Histogram of the\n Frangi image')

# Display the histogram of the equalized image
plt.subplot(3, 2, 6)
plt.hist(equalized_frangi.flatten(), bins=256, range=[0, 256], color='black')
plt.title(f'Histogram of the\n equalized image')

plt.tight_layout()
plt.show()